In [13]:
import os
import json
import numpy as np
from omegaconf import OmegaConf

In [14]:
exp_dir = "../../../logs/en/asr/evaluate"
folders = {
    "Fine-tuned_9": "version_9",
    "AC_10": "version_43",
    "AC_standard": "version_24",
    "AC_0.7": "version_25",
    "AC_0.5": "version_26",
    "AC_0.3": "version_27",
    "AC_0.1": "version_19",
    "AC_0.01": "version_38",
}

In [15]:
# define which accent is standard, which are seen and which are unseen
classes = {
    "seen": ["us", "uk", "ca", "in", "de", "au"],
    "unseen": ["hk", "sg", "sc", "nz", "ie", "za", "ni", "ph"],
}

mapping = {
    "United States English": "us",
    "England English": "uk",
    "India and South Asia (India, Pakistan, Sri Lanka)": "in",
    "Canadian English": "ca",
    "Australian English": "au",
    "German English,Non native speaker": "de",
    "Scottish English": "sc",
    "New Zealand English": "nz",
    "Irish English": "ie",
    "Southern African (South Africa, Zimbabwe, Namibia)": "za",
    "Northern Irish": "ni",
    "Filipino": "ph",
    "Hong Kong English": "hk",
    "Singaporean English": "sg",
}

In [16]:
# load WERs of each file, for each accent
wers = dict()
for exp, folder in folders.items():
    wers[exp] = {k[5:-4]: v for k, v in json.load(open(f"{exp_dir}/{folder}/avg_wers.json")).items()}

In [17]:
# transform dictionary into a numpy array, where one axis represents the accents and another the experiments
experiments = list(wers.keys())
accents = list(wers[experiments[0]].keys())
n_words = np.array([wers[experiments[0]][acc]["n_words"] for acc in accents])
avg_wers = np.array([[wers[exp][acc]["avg_wer"] for exp in experiments] for acc in accents])

In [18]:
# compute means for each class, for each experiment
class_indices = {k: [accents.index(acc) for acc in v] for k, v in classes.items()}
class_means = {k: np.mean(avg_wers[indices], axis=0) for k, indices in class_indices.items()}

In [19]:
# print the avg. WERs and the means (overall and per class) as a markdown table
headers = ["Accent / Dialect"] + experiments
print((" | ").join(headers))
print(f"|{'---:|'*len(headers)}")

# print avg. WERs
for i in range(len(accents)):
    row = [list(mapping.keys())[list(mapping.values()).index(accents[i])]] + [f"{avg_wers[i,j]*100:.2f}" for j in range(len(experiments))]
    print((" | ").join(row))

# print overall means
row = ["mean"] + [f"{np.mean(avg_wers[:,i])*100:.2f}" for i in range(len(experiments))]
print((" | ").join(row))

# print class means
for key, value in class_means.items():
    row = [f"{key} mean"] + [f"{value[i]*100:.2f}" for i in range(len(experiments))]
    print((" | ").join(row))

# print worst avg. WER of each experiment
row = ["worst"] + [f"{v*100:.2f}" for v in np.max(avg_wers, axis=0)]
print((" | ").join(row))

# print the avg. of the three worst avg. WERs of each experiment
row = ["3-worst mean."] + [f"{np.mean(np.sort(avg_wers, axis=0)[-3:,i])*100:.2f}" for i in range(len(experiments))]
print((" | ").join(row))


Accent / Dialect | Fine-tuned_9 | AC_10 | AC_standard | AC_0.7 | AC_0.5 | AC_0.3 | AC_0.1 | AC_0.01
|---:|---:|---:|---:|---:|---:|---:|---:|---:|
Southern African (South Africa, Zimbabwe, Namibia) | 6.54 | 7.53 | 6.54 | 6.49 | 6.53 | 6.51 | 6.53 | 7.92
Singaporean English | 10.41 | 12.39 | 10.51 | 10.63 | 10.39 | 10.65 | 10.41 | 13.18
India and South Asia (India, Pakistan, Sri Lanka) | 9.21 | 10.84 | 9.41 | 9.25 | 9.31 | 9.21 | 9.21 | 10.95
Australian English | 7.47 | 8.50 | 7.53 | 7.47 | 7.54 | 7.44 | 7.46 | 9.08
Hong Kong English | 9.76 | 12.19 | 9.75 | 9.93 | 9.75 | 9.97 | 9.78 | 12.15
Irish English | 7.51 | 9.17 | 7.41 | 7.51 | 7.37 | 7.52 | 7.52 | 9.41
England English | 5.78 | 7.02 | 5.87 | 5.79 | 5.83 | 5.75 | 5.78 | 7.09
Filipino | 8.90 | 10.28 | 8.79 | 8.99 | 8.78 | 8.95 | 8.91 | 10.26
German English,Non native speaker | 5.62 | 6.72 | 5.66 | 5.60 | 5.60 | 5.58 | 5.62 | 6.74
Northern Irish | 6.96 | 8.02 | 7.02 | 6.71 | 7.05 | 6.72 | 6.96 | 8.00
United States English | 5.43 | 6.

Accent / Dialect | Fine-tuned_9 | AC_10 | AC_standard | AC_0.7 | AC_0.5 | AC_0.3 | AC_0.1 | AC_0.01
|---:|---:|---:|---:|---:|---:|---:|---:|---:|
Southern African (South Africa, Zimbabwe, Namibia) | 6.54 | 7.53 | 6.54 | 6.49 | 6.53 | 6.51 | 6.53 | 7.92
Singaporean English | 10.41 | 12.39 | 10.51 | 10.63 | 10.39 | 10.65 | 10.41 | 13.18
India and South Asia (India, Pakistan, Sri Lanka) | 9.21 | 10.84 | 9.41 | 9.25 | 9.31 | 9.21 | 9.21 | 10.95
Australian English | 7.47 | 8.50 | 7.53 | 7.47 | 7.54 | 7.44 | 7.46 | 9.08
Hong Kong English | 9.76 | 12.19 | 9.75 | 9.93 | 9.75 | 9.97 | 9.78 | 12.15
Irish English | 7.51 | 9.17 | 7.41 | 7.51 | 7.37 | 7.52 | 7.52 | 9.41
England English | 5.78 | 7.02 | 5.87 | 5.79 | 5.83 | 5.75 | 5.78 | 7.09
Filipino | 8.90 | 10.28 | 8.79 | 8.99 | 8.78 | 8.95 | 8.91 | 10.26
German English,Non native speaker | 5.62 | 6.72 | 5.66 | 5.60 | 5.60 | 5.58 | 5.62 | 6.74
Northern Irish | 6.96 | 8.02 | 7.02 | 6.71 | 7.05 | 6.72 | 6.96 | 8.00
United States English | 5.43 | 6.38 | 5.46 | 5.54 | 5.45 | 5.52 | 5.43 | 6.31
Canadian English | 5.36 | 6.10 | 5.35 | 5.46 | 5.34 | 5.46 | 5.36 | 6.12
New Zealand English | 5.40 | 6.61 | 5.28 | 5.54 | 5.34 | 5.56 | 5.39 | 7.07
Scottish English | 33.50 | 36.96 | 32.55 | 33.43 | 33.21 | 33.77 | 33.51 | 36.51
mean | 9.13 | 10.62 | 9.08 | 9.17 | 9.11 | 9.19 | 9.13 | 10.77
seen mean | 6.48 | 7.59 | 6.55 | 6.52 | 6.51 | 6.49 | 6.48 | 7.72
unseen mean | 11.12 | 12.89 | 10.98 | 11.15 | 11.05 | 11.21 | 11.13 | 13.06
worst | 33.50 | 36.96 | 32.55 | 33.43 | 33.21 | 33.77 | 33.51 | 36.51
3-worst mean. | 17.89 | 20.51 | 17.61 | 18.00 | 17.78 | 18.13 | 17.90 | 20.61

In [20]:
# print the experiment folders of each experiment (both train and eval folders)
print("Experiment folders:\n")
for exp, folder in folders.items():
    eval_folder = os.path.join(exp_dir, folder)
    eval_config = OmegaConf.load(os.path.join(eval_folder, "config.yaml"))
    train_folder = f'../{eval_config.asr.ckpt.replace("/checkpoints/last.ckpt", "")}'
    print(f"- {exp}: training `{train_folder[3:]}`, evaluation `{eval_folder[3:]}`")

Experiment folders:

- Fine-tuned_9: training `logs/en/asr/train/version_11`, evaluation `../../logs/en/asr/evaluate/version_9`
- AC_10: training `logs/en/ensemble/train/binary/b7/DAT/version_85`, evaluation `../../logs/en/asr/evaluate/version_43`
- AC_standard: training `logs/en/ensemble/train/binary/b7/DAT/version_66`, evaluation `../../logs/en/asr/evaluate/version_24`
- AC_0.7: training `logs/en/ensemble/train/binary/b7/DAT/version_67`, evaluation `../../logs/en/asr/evaluate/version_25`
- AC_0.5: training `logs/en/ensemble/train/binary/b7/DAT/version_68`, evaluation `../../logs/en/asr/evaluate/version_26`
- AC_0.3: training `logs/en/ensemble/train/binary/b7/DAT/version_69`, evaluation `../../logs/en/asr/evaluate/version_27`
- AC_0.1: training `logs/en/ensemble/train/binary/b7/DAT/version_64`, evaluation `../../logs/en/asr/evaluate/version_19`
- AC_0.01: training `logs/en/ensemble/train/binary/b7/DAT/version_80`, evaluation `../../logs/en/asr/evaluate/version_38`
